# 🚀 AlphaBot ML/DL Training - Google Colab

Notebook complet pour entraîner les modèles ML/DL d'AlphaBot sous Colab (GPU L4 ou TPU), avec suivi/reprise, téléchargement de données robuste, et fallbacks sûrs.

In [ ]:
# Ajustement du path pour que Colab trouve le module alphabot
import sys
sys.path.append('/content')
sys.path.append('/content/alphabot')

## 🔄 Suivi de Progression et Reprise Automatique

In [ ]:
# 🔄 Système de suivi et reprise automatique
import os
import json
from datetime import datetime

base_path = '/content/drive/MyDrive/AlphaBot_ML_Training'
os.makedirs(base_path, exist_ok=True)
progress_file = f'{base_path}/progress_tracker.json'

default_progress = {
    'cell_1_setup': False,
    'cell_2_data_download': False,
    'cell_3_data_analysis': False,
    'cell_4_pattern_training': False,
    'cell_5_sentiment_training': False,
    'cell_6_rag_training': False,
    'cell_7_integration': False,
    'cell_8_testing': False,
    'cell_9_deployment': False,
    'last_cell_executed': None,
    'start_time': None,
    'last_update': None
}

try:
    with open(progress_file, 'r') as f:
        progress = json.load(f)
    print('📊 Suivi de progression chargé')
except Exception:
    progress = default_progress.copy()
    progress['start_time'] = datetime.now().isoformat()
    print('🆕 Nouveau suivi de progression initialisé')

def update_progress(cell_name):
    progress.setdefault(cell_name, False)
    progress[cell_name] = True
    progress['last_cell_executed'] = cell_name
    progress['last_update'] = datetime.now().isoformat()
    with open(progress_file, 'w') as f:
        json.dump(progress, f, indent=2)
    print(f'✅ Progression mise à jour: {cell_name}')

def check_progress():
    print('\n📋 État actuel de la progression:')
    print('=' * 50)
    completed = sum(1 for k, v in progress.items() if k.startswith('cell_') and isinstance(v, bool) and v)
    total = sum(1 for k in default_progress.keys() if k.startswith('cell_'))
    pct = (completed / total * 100) if total else 0.0
    print(f'📊 Progression: {completed}/{total} étapes complétées ({pct:.1f}%)')
    print(f'⏰ Démarré: {progress.get("start_time", "N/A")}')
    print(f'🔄 Dernière mise à jour: {progress.get("last_update", "N/A")}')
    print(f'📍 Dernière cellule: {progress.get("last_cell_executed", "Aucune")}')
    print('\n📝 Statut des étapes:')
    steps = [
        ('cell_1_setup', '1. Configuration initiale'),
        ('cell_2_data_download', '2. Téléchargement des données'),
        ('cell_3_data_analysis', '3. Analyse des données'),
        ('cell_4_pattern_training', '4. Entraînement Pattern Detector'),
        ('cell_5_sentiment_training', '5. Entraînement Sentiment Analyzer'),
        ('cell_6_rag_training', '6. Entraînement RAG'),
        ('cell_7_integration', '7. Intégration'),
        ('cell_8_testing', '8. Tests'),
        ('cell_9_deployment', '9. Déploiement')
    ]
    for step_key, step_name in steps:
        status = '✅' if progress.get(step_key, False) else '⏳'
        print(f'  {status} {step_name}')
    print('=' * 50)
    if not progress.get('cell_1_setup', False):
        print('\n🚀 Prochaine étape: Exécuter la cellule 1 (Configuration)')
    elif not progress.get('cell_2_data_download', False):
        print('\n🚀 Prochaine étape: Exécuter la cellule 2 (Téléchargement des données)')
    elif not progress.get('cell_3_data_analysis', False):
        print('\n🚀 Prochaine étape: Exécuter la cellule 3 (Analyse des données)')
    elif not progress.get('cell_4_pattern_training', False):
        print('\n🚀 Prochaine étape: Exécuter la cellule 4 (Pattern Detector)')
    elif not progress.get('cell_5_sentiment_training', False):
        print('\n🚀 Prochaine étape: Exécuter la cellule 5 (Sentiment Analyzer)')
    elif not progress.get('cell_6_rag_training', False):
        print('\n🚀 Prochaine étape: Exécuter la cellule 6 (RAG)')
    elif not progress.get('cell_7_integration', False):
        print('\n🚀 Prochaine étape: Exécuter la cellule 7 (Intégration)')
    elif not progress.get('cell_8_testing', False):
        print('\n🚀 Prochaine étape: Exécuter la cellule 8 (Tests)')
    elif not progress.get('cell_9_deployment', False):
        print('\n🚀 Prochaine étape: Exécuter la cellule 9 (Déploiement)')
    else:
        print('\n🎉 Toutes les étapes sont complétées !')

check_progress()
print('\n💡 Instructions:')
print("1. Exécutez cette cellule pour voir l'état d'avancement")
print('2. Chaque cellule mettra à jour automatiquement sa progression')
print("3. Si le processus s'arrête, relancez simplement cette cellule")
print('4. Continuez avec la cellule suggérée')
print('\n🔄 Note: Le système est conçu pour supporter les arrêts/redémarrages')

## 1) Setup GPU/TPU et environnement Colab

In [ ]:
import tensorflow as tf
import torch, logging
logging.basicConfig(level=logging.INFO)

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    print('✅ TPU détectée et configurée')
except Exception:
    try:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            strategy = tf.distribute.MirroredStrategy()
            print(f'✅ {len(gpus)} GPU(s) détectée(s)')
        else:
            strategy = tf.distribute.get_strategy()
            print('⚠️ Aucun GPU/TPU détecté, utilisation du CPU')
    except Exception as e:
        strategy = tf.distribute.get_strategy()
        print(f'⚠️ Erreur de configuration GPU: {e}')

try:
    policy = tf.keras.mixed_precision.Policy('mixed_float16')
    tf.keras.mixed_precision.set_global_policy(policy)
    print('✅ Mixed precision activée')
except Exception:
    print('⚠️ Mixed precision non disponible')

print('\n📊 Configuration:')
print(f'- TensorFlow: {tf.__version__}')
print(f'- PyTorch: {torch.__version__}')
print(f'- Strategy: {strategy}')
print(f"- GPUs disponibles: {tf.config.list_physical_devices('GPU')}")
if torch.cuda.is_available():
    print(f'- CUDA: {torch.version.cuda}')
    print(f'- GPU: {torch.cuda.get_device_name(0)}')
update_progress('cell_1_setup')

## 2) Montage Google Drive (résilient v2)

In [ ]:
print('🔧 Configuration Google Drive (résiliente v2)...')
from google.colab import drive
import os, shutil, time
MOUNT_POINT = '/content/drive'

def _safe_cleanup_mount_point(mp: str):
    try:
        if os.path.islink(mp): os.unlink(mp)
        if os.path.isdir(mp):
            for entry in os.listdir(mp):
                p = os.path.join(mp, entry)
                try:
                    if os.path.isfile(p) or os.path.islink(p): os.remove(p)
                    elif os.path.isdir(p): shutil.rmtree(p)
                except Exception: pass
        else:
            os.makedirs(mp, exist_ok=True)
    except Exception as e:
        print(f'⚠️ Nettoyage mount point: {e}')

def _force_unmount():
    try: drive.flush_and_unmount()
    except Exception: pass
    try:
        os.system('fusermount -u /content/drive 2>/dev/null || true')
        os.system('umount /content/drive 2>/dev/null || true')
    except Exception: pass

_force_unmount(); time.sleep(1)
_safe_cleanup_mount_point(MOUNT_POINT); time.sleep(0.5)
try:
    drive.mount(MOUNT_POINT, force_remount=True)
    print('✅ Drive monté (v2)')
except Exception as e:
    print(f'❌ drive.mount a échoué: {e}')
    if 'Mountpoint must not already contain files' in str(e):
        try:
            shutil.rmtree(MOUNT_POINT, ignore_errors=True)
            os.makedirs(MOUNT_POINT, exist_ok=True)
            drive.mount(MOUNT_POINT, force_remount=True)
            print('✅ Drive monté après recréation du dossier')
        except Exception as e2:
            print(f'⚠️ Impossible de recréer {MOUNT_POINT}: {e2}')
            raise
base_path = '/content/drive/MyDrive/AlphaBot_ML_Training'
for sub in ('data', 'models', 'checkpoints', 'logs', 'exports'):
    os.makedirs(f'{base_path}/{sub}', exist_ok=True)
print(f'📁 Répertoires prêts sous: {base_path}')
update_progress('cell_2_data_download')

## 3) Setup du dépôt AlphaBot et dépendances Colab

In [ ]:
import subprocess, sys
from pathlib import Path
if not Path('/content/alphabot').exists():
    print('📥 Clonage du dépôt AlphaBot...')
    subprocess.run(['git', 'clone', 'https://github.com/thomy03/alphabot.git', '/content/alphabot'], check=True)
else:
    print('📂 Dépôt AlphaBot déjà présent')
print('📦 Installation des dépendances...')
try:
    subprocess.run(['pip', 'install', '-r', '/content/alphabot/requirements_colab.txt'], check=True)
    print('✅ Dépendances installées')
except subprocess.CalledProcessError as e:
    print(f'⚠️ Erreur requirements_colab.txt: {e}')
    for pkg in ['tensorflow','torch','transformers','sentence-transformers','faiss-cpu','yfinance','pandas','numpy','scikit-learn','matplotlib','seaborn','tqdm','requests']:
        try:
            subprocess.run(['pip', 'install', pkg], check=True)
            print(f'✅ {pkg} installé')
        except Exception as e2:
            print(f'❌ {pkg} échec: {e2}')
sys.path.append('/content'); sys.path.append('/content/alphabot')
update_progress('cell_3_data_analysis')

## 4) Téléchargement des données (robuste)

In [ ]:
import yfinance as yf, pandas as pd, numpy as np
from datetime import datetime, timedelta
import time, pickle, os
# Étendre la période et la liste de symboles pour augmenter la couverture
symbols = ['AAPL','GOOGL','MSFT','TSLA','AMZN','NVDA','META','NFLX','IBM','ORCL','INTC','AMD','QCOM','CRM','SAP','CSCO','JPM','BAC','V','MA','DIS','KO','PEP','NKE','XOM']
end_date = datetime.now(); start_date = end_date - timedelta(days=365*5)  # 5 ans
print(f'📥 Téléchargement pour {symbols} sur 5 ans...')
def safe_download(symbol, start, end, tries=3, sleep_s=1.0):
    last_err=None
    for _ in range(tries):
        try:
            df = yf.download(symbol, start=start, end=end, auto_adjust=True)
            if df is not None and not df.empty: return df
        except Exception as e: last_err=e
        time.sleep(sleep_s)
    if last_err: print(f'❌ {symbol}: {last_err}')
    return None
all_data={}
for s in symbols:
    d=safe_download(s,start_date,end_date)
    if d is not None and not d.empty:
        all_data[s]=d; print(f'✅ {s}: {len(d)} jours')
    else:
        print(f'❌ {s}: vide')
data_path=f'{base_path}/data/market_data.pkl'
os.makedirs(f'{base_path}/data',exist_ok=True)
with open(data_path,'wb') as f: pickle.dump(all_data,f)
print(f'💾 Données sauvegardées: {data_path}')
update_progress('cell_4_pattern_training')

## 5) Entraînement Pattern Detector — L4 GPU non-cuDNN + préparation de données robuste

In [ ]:
import os
os.environ['TF_KERAS_ALLOW_CUDNN_RNN']='0'  # forcer chemin non-cuDNN
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import numpy as np, pickle, pandas as pd

# Debug logs
log_path = f"{base_path}/logs"
os.makedirs(log_path, exist_ok=True)
log_file = f"{log_path}/pattern_debug.txt"
open(log_file,'a').write(f"\n==== DEBUG RUN {datetime.now().isoformat()} ====\n")

try:
    with open(f'{base_path}/data/market_data.pkl','rb') as f:
        all_data = pickle.load(f)
    print('✅ Données chargées')
except Exception:
    print('❌ Données introuvables — exécutez la cellule 4')

def prepare_pattern_training_data(all_data):
    def normalize_yf_cols(df: pd.DataFrame):
        df = df.copy()
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = ['_'.join([str(c) for c in col if c is not None]) for col in df.columns]
        else:
            df.columns = [str(c) for c in df.columns]
        cols_lower = {c.lower().replace(' ', '').replace('-', '_'): c for c in df.columns}
        def pick(*aliases):
            for a in aliases:
                key = a.lower().replace(' ', '').replace('-', '_')
                if key in cols_lower: return cols_lower[key]
            return None
        close = pick('Close','Adj Close','Adj_Close','CloseAdjClose','close')
        high  = pick('High','high')
        low   = pick('Low','low')
        vol   = pick('Volume','volume')
        if vol is None and close is not None:
            ret = df[close].pct_change()
            df['Volume_proxy'] = ret.rolling(10, min_periods=3).std().fillna(0.0)
            vol = 'Volume_proxy'
        for c in [close, high, low, vol]:
            if c is not None:
                df[c] = pd.to_numeric(df[c], errors='coerce')
                df[c] = df[c].replace([np.inf, -np.inf], np.nan)
                df[c] = df[c].interpolate(limit_direction='both')
                df[c] = df[c].fillna(method='bfill').fillna(method='ffill')
        open(log_file,'a').write(f"cols: close={close}, high={high}, low={low}, vol={vol}\n")
        return df, close, high, low, vol

    X, y = [], []
    for symbol, data in all_data.items():
        try:
            if data is None or not hasattr(data, 'empty') or data.empty:
                continue
            df, close, hi, lo, vol = normalize_yf_cols(data.sort_index())
            if not all([close, hi, lo, vol]):
                continue
            df = df.dropna(subset=[close, hi, lo, vol])
            n = len(df)
            if n < 23:
                continue
            win_count = 0
            open(log_file,'a').write(f"symbol={symbol} len={n} after_dropna\n")
            for i in range(0, n - 22):
                seq = df.iloc[i:i+20]
                fut = df.iloc[i+20:i+23]
                if seq[[close, vol, hi, lo]].isnull().any().any() or fut[[close]].isnull().any().any():
                    continue
                close_arr  = np.asarray(seq[close].values, dtype=np.float32).reshape(-1, 1)
                vol_arr    = np.asarray(seq[vol].values, dtype=np.float32).reshape(-1, 1)
                spread_arr = np.asarray((seq[hi] - seq[lo]).values, dtype=np.float32).reshape(-1, 1)
                features = np.concatenate([close_arr, vol_arr, spread_arr], axis=1)
                if features.shape != (20, 3):
                    continue
                current_price = float(seq[close].iloc[-1])
                if not np.isfinite(current_price) or current_price <= 0:
                    continue
                future_mean = float(np.mean(fut[close].values))
                if not np.isfinite(future_mean):
                    continue
                future_return = (future_mean - current_price) / current_price
                label = 2 if future_return > 0.02 else (0 if future_return < -0.02 else 1)
                X.append(features); y.append(label)
                win_count += 1
        except Exception as e:
            open(log_file,'a').write(f"symbol={symbol} ERROR {e}\n")
            continue
    open(log_file,'a').write(f"total_windows={len(X)} by_symbol_last={win_count}\n")
    X = np.array(X, dtype=np.float32)
    Y = np.array(y, dtype=np.int32)
    print(f'✅ Préparation: X={X.shape}, y={Y.shape}')
    print(f"🔍 Log fenêtres: voir {log_file}")
    return X, Y

X_train, y_train = prepare_pattern_training_data(all_data)
print(f'📊 Échantillons: {X_train.shape[0]}')

strategy = tf.distribute.get_strategy()
with strategy.scope():
    inputs = tf.keras.Input(shape=(20,3), name='input')
    x = tf.keras.layers.BatchNormalization()(inputs)
    x = tf.keras.layers.LSTM(64, return_sequences=False, activation='tanh', recurrent_activation='sigmoid',
                             use_bias=True, unit_forget_bias=True, unroll=False,
                             dropout=0.1, recurrent_dropout=0.1,
                             kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal',
                             name='lstm_main_ncudnn')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    outputs = tf.keras.layers.Dense(3, activation='softmax', name='output')(x)
    model = tf.keras.Model(inputs, outputs, name='l4_ncudnn_lstm_model')
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print('✅ Modèle L4 (non-cuDNN) créé')

X_train = X_train.astype(np.float32); y_train = y_train.astype(np.int32)
if X_train.shape[0] == 0:
    print('⚠️ Dataset vide. Génération synthétique minimale...')
    X_train = np.random.randn(1024, 20, 3).astype(np.float32)
    y_train = np.random.randint(0, 3, size=(1024,)).astype(np.int32)

scaler = (StandardScaler().fit(X_train.reshape(-1,3)))
Xs = scaler.transform(X_train.reshape(-1,3)).reshape(X_train.shape)
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, monitor='val_loss', verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3, monitor='val_loss', verbose=1)
]
try:
    hist = model.fit(Xs, y_train, epochs=10, batch_size=64, validation_split=0.2, callbacks=callbacks, verbose=1)
    print('✅ Entraînement terminé')
except Exception as e:
    print(f'❌ Erreur entraînement (GPU non-cuDNN). Fallback Dense-only: {e}')
    with tf.distribute.get_strategy().scope():
        cpu_model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(20,3)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation='relu'), tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(64, activation='relu'),  tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(32, activation='relu'),  tf.keras.layers.Dense(3, activation='softmax')
        ])
        cpu_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    hist = cpu_model.fit(Xs, y_train, epochs=8, batch_size=64, validation_split=0.2, verbose=1)
    model = cpu_model
    print('✅ Modèle Dense-only entraîné')

import os, pickle, matplotlib.pyplot as plt
os.makedirs(f'{base_path}/models', exist_ok=True)
model.save(f'{base_path}/models/pattern_model_l4_ncudnn.keras')
with open(f'{base_path}/models/pattern_scaler.pkl','wb') as f: pickle.dump(scaler,f)
print('💾 Modèle/scaler sauvegardés')
try:
    plt.figure(figsize=(12,4));
    if 'accuracy' in hist.history: plt.subplot(1,2,1); plt.plot(hist.history['accuracy']); 
    if 'val_accuracy' in hist.history: plt.plot(hist.history['val_accuracy']); plt.title('Accuracy'); plt.legend(['train','val'])
    if 'loss' in hist.history: plt.subplot(1,2,2); plt.plot(hist.history['loss']);
    if 'val_loss' in hist.history: plt.plot(hist.history['val_loss']); plt.title('Loss'); plt.legend(['train','val']); plt.tight_layout(); plt.show()
except Exception:
    pass
update_progress('cell_5_sentiment_training')

## 6) Entraînement Sentiment Analyzer (FinBERT) — correctif torch_xla/_XLAC pour GPU L4 (renforcé)

In [ ]:
# Correctif renforcé: purge TPU/XLA en mémoire, désinstalle torch-xla, pin torch/transformers, puis import propre
import os, sys, subprocess, importlib
os.environ.pop('COLAB_TPU_ADDR', None)
for mod in list(sys.modules.keys()):
    if mod.startswith('torch_xla') or mod == '_XLAC':
        sys.modules.pop(mod, None)
subprocess.run(['pip', 'uninstall', '-y', 'torch-xla'], check=False)
subprocess.run(['pip', 'install', '-q', '--upgrade', 'torch==2.2.2', 'transformers==4.41.2'], check=False)
importlib.invalidate_caches()

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts=texts; self.labels=labels; self.tokenizer=tokenizer; self.max_length=max_length
    def __len__(self): return len(self.texts)
    def __getitem__(self, idx):
        t=str(self.texts[idx]); l=self.labels[idx]
        enc=self.tokenizer(t,truncation=True,padding='max_length',max_length=self.max_length,return_tensors='pt')
        return {'input_ids':enc['input_ids'].flatten(),'attention_mask':enc['attention_mask'].flatten(),'labels':torch.tensor(l)}

texts=[
    'Apple reports record quarterly earnings',
    'Google stock drops on regulatory concerns',
    'Tesla announces new battery technology',
    'Microsoft cloud growth exceeds expectations',
    'Amazon faces antitrust investigation']
labels=[2,0,2,2,0]
model_name='yiyanghkust/finbert-tone'
tok=AutoTokenizer.from_pretrained(model_name)
mdl=AutoModelForSequenceClassification.from_pretrained(model_name)
ds=SentimentDataset(texts,labels,tok)
args=TrainingArguments(output_dir=f'{base_path}/checkpoints/sentiment',num_train_epochs=1,per_device_train_batch_size=4,
                       logging_dir=f'{base_path}/logs/sentiment',logging_steps=1,save_strategy='no',evaluation_strategy='no')
trainer=Trainer(model=mdl,args=args,train_dataset=ds,tokenizer=tok)
print('🚀 Entraînement FinBERT...'); trainer.train()
os.makedirs(f'{base_path}/models',exist_ok=True)
mdl.save_pretrained(f'{base_path}/models/finbert_sentiment'); tok.save_pretrained(f'{base_path}/models/finbert_sentiment')
print('✅ FinBERT sauvegardé')
update_progress('cell_6_rag_training')

## 7) RAG Integrator (Embeddings + FAISS)

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss, numpy as np, pickle, os
docs=[
 'Apple Inc. is a technology company...',
 'Alphabet Inc. provides various products...',
 'Microsoft Corporation develops software...',
 'Amazon.com, Inc. engages in retail...',
 'Tesla, Inc. designs electric vehicles...']
emb=SentenceTransformer('all-MiniLM-L6-v2')
vecs=emb.encode(docs)
index=faiss.IndexFlatL2(vecs.shape[1]); index.add(vecs)
faiss.write_index(index,f'{base_path}/models/faiss_index.bin')
with open(f'{base_path}/models/documents.pkl','wb') as f: pickle.dump(docs,f)
print('✅ Index/documents sauvegardés')
update_progress('cell_7_integration')

## 8) Intégration et tests rapides

In [ ]:
import json, numpy as np, pickle
from datetime import datetime
print('🔧 Intégration finale...')
try:
    pm=tf.keras.models.load_model(f'{base_path}/models/pattern_model_l4_ncudnn.keras')
    print('✅ Pattern model chargé')
    from transformers import AutoTokenizer, AutoModelForSequenceClassification
    st=AutoTokenizer.from_pretrained(f'{base_path}/models/finbert_sentiment')
    sm=AutoModelForSequenceClassification.from_pretrained(f'{base_path}/models/finbert_sentiment')
    print('✅ FinBERT chargé')
    import faiss
    fx=faiss.read_index(f'{base_path}/models/faiss_index.bin')
    with open(f'{base_path}/models/documents.pkl','rb') as f: DD=pickle.load(f)
    print('✅ FAISS/documents chargés')
except Exception as e:
    print(f'❌ Erreur chargement: {e}')

report={'timestamp': datetime.now().isoformat(),'models_trained':{'pattern':'ok','sentiment':'ok','rag':'ok'}}
os.makedirs(f'{base_path}/exports',exist_ok=True)
rp=f'{base_path}/exports/performance_report.json'
with open(rp,'w') as f: json.dump(report,f,indent=2)
print(f'📊 Rapport sauvegardé: {rp}')
update_progress('cell_8_testing')
print('✅ Pipeline terminé')